In [103]:
import pandas as pd
from map4 import MAP4Calculator
from rdkit import Chem
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import tmap as tm
from faerun import Faerun
from matplotlib.colors import ListedColormap

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
glass = pd.read_csv("ligands.tsv", sep="\t")

In [90]:
suppl = Chem.SDMolSupplier(str("P08173.sdf"))
df_rows = []
for mol in suppl:
    if mol is not None:
        inchi = mol.GetProp("InChI Key")
        smiles = Chem.MolToSmiles(mol)
        df_rows.append([inchi,smiles,mol])
mol_df = pd.DataFrame(df_rows, columns=('InChI Key', "smiles",'mol'))

In [91]:
M4_ligands = mol_df['InChI Key'].to_list()

In [92]:
def M4_ligand(inchi):
    if inchi in M4_ligands:
        return 1
    else:
        return 0

In [93]:
glass["M4_ligand"] = glass["InChI Key"].map(M4_ligand)

In [94]:
glass.query("M4_ligand == 1")

,Ligand Name,CID,Molecular Formula,Molecular Weight,IUPAC Name,Canonical SMILES,Isomeric SMILES,InChI Std. ID,InChI Key,XlogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,MAP4,label,M4_ligand
192,CHEMBL24369,15157141,C24H39NO2,373.581,5-(diethylamino)pentyl 2-cyclohexyl-2-phenylpr...,CCN(CC)CCCCCOC(=O)C(C)(C1CCCCC1)C2=CC=CC=C2,CCN(CC)CCCCCOC(=O)C(C)(C1CCCCC1)C2=CC=CC=C2,InChI=1S/C24H39NO2/c1-4-25(5-2)19-13-8-14-20-2...,SHORJVPMMAZEFL-UHFFFAOYSA-N,6.7,0,3,"[6670835, 2349897, 5230804, 5394757, 23381705,...",CCN(CC)CCCCCOC(=O)C(C)(C1CCCCC1)C2=CC=CC=C2__<...,1
840,CHEMBL120599,44346318,C24H33NO6,431.529,"2-cyclohexylethyl 1-ethyl-4-phenyl-3,6-dihydro...",CCN1CC=C(C(C1)C(=O)OCCC2CCCCC2)C3=CC=CC=C3.C(=...,CCN1CC=C(C(C1)C(=O)OCCC2CCCCC2)C3=CC=CC=C3.C(=...,InChI=1S/C22H31NO2.C2H2O4/c1-2-23-15-13-20(19-...,OODYHGDZYKNGBL-UHFFFAOYSA-N,NaN,2,7,"[17834397, 2833729, 517419, 8936333, 30693471,...",CCN1CC=C(C(C1)C(=O)OCCC2CCCCC2)C3=CC=CC=C3.C(=...,1
909,CHEMBL3310918,73671019,C25H37N5O3,455.603,ethyl 4-[[1-(pyrrolidine-1-carbonyl)spiro[2H-p...,CCOC(=O)N1CCC(CC1)CN2CCC3(CC2)CN(C4=C3C=CC=N4)...,CCOC(=O)N1CCC(CC1)CN2CCC3(CC2)CN(C4=C3C=CC=N4)...,InChI=1S/C25H37N5O3/c1-2-33-24(32)29-14-7-20(8...,ZVUHRPULMOACBY-UHFFFAOYSA-N,2.4,0,5,"[18903355, 81933, 27603635, 1179055, 6982840, ...",CCOC(=O)N1CCC(CC1)CN2CCC3(CC2)CN(C4=C3C=CC=N4)...,1
948,CHEMBL594376,46227554,C9H20INO2,301.168,"trimethyl-[[(2R,5R)-5-methyl-1,4-dioxan-2-yl]m...",CC1COC(CO1)C[N+](C)(C)C.[I-],C[C@@H]1CO[C@@H](CO1)C[N+](C)(C)C.[I-],InChI=1S/C9H20NO2.HI/c1-8-6-12-9(7-11-8)5-10(2...,MMFSFQDCALVVNK-VTLYIQCISA-M,NaN,0,3,"[26176407, 28878746, 12434627, 24246647, 38389...",C[C@@H]1CO[C@@H](CO1)C[N+](C)(C)C.[I-]__<a tar...,1
1638,R-dimethindene,12963076,C20H24N2,292.426,"N,N-dimethyl-2-[3-[(1R)-1-pyridin-2-ylethyl]-1...",CC(C1=CC=CC=N1)C2=C(CC3=CC=CC=C32)CCN(C)C,C[C@@H](C1=CC=CC=N1)C2=C(CC3=CC=CC=C32)CCN(C)C,InChI=1S/C20H24N2/c1-15(19-10-6-7-12-21-19)20-...,MVMQESMQSYOVGV-HNNXBMFYSA-N,2.7,0,2,"[33201605, 2843844, 16021802, 45822764, 320906...",C[C@@H](C1=CC=CC=N1)C2=C(CC3=CC=CC=C32)CCN(C)C...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341744,CHEMBL491208,44579416,C25H28ClN5O,449.983,2-[[4-[4-(5-chloro-2-oxo-3H-benzimidazol-1-yl)...,C1CN(CCC1N2CCC(CC2)N3C4=C(C=C(C=C4)Cl)NC3=O)CC...,C1CN(CCC1N2CCC(CC2)N3C4=C(C=C(C=C4)Cl)NC3=O)CC...,InChI=1S/C25H28ClN5O/c26-20-5-6-24-23(15-20)28...,FVBQJCJEQPAJFJ-UHFFFAOYSA-N,3.8,1,4,"[9845650, 3827191, 2187645, 4229638, 2852937, ...",C1CN(CCC1N2CCC(CC2)N3C4=C(C=C(C=C4)Cl)NC3=O)CC...,1
341793,amisulpride,2159,C17H27N3O4S,369.480,4-amino-N-[(1-ethylpyrrolidin-2-yl)methyl]-5-e...,CCN1CCCC1CNC(=O)C2=CC(=C(C=C2OC)N)S(=O)(=O)CC,CCN1CCCC1CNC(=O)C2=CC(=C(C=C2OC)N)S(=O)(=O)CC,InChI=1S/C17H27N3O4S/c1-4-20-8-6-7-12(20)11-19...,NTJOBXMMWNYJFB-UHFFFAOYSA-N,1.5,2,6,"[371106, 9090649, 2638719, 599760, 1400336, 19...",CCN1CCCC1CNC(=O)C2=CC(=C(C=C2OC)N)S(=O)(=O)CC_...,1
342131,CID 559589,559589,C19H32BrNO2,386.374,diethyl-methyl-[2-(3-methyl-2-phenylpentanoyl)...,CCC(C)C(C1=CC=CC=C1)C(=O)OCC[N+](C)(CC)CC.[Br-],CCC(C)C(C1=CC=CC=C1)C(=O)OCC[N+](C)(CC)CC.[Br-],InChI=1S/C19H32NO2.BrH/c1-6-16(4)18(17-12-10-9...,CEJGGHKJHDHLAZ-UHFFFAOYSA-M,NaN,0,3,"[6670835, 2349897, 5230804, 7597000, 6420407, ...",CCC(C)C(C1=CC=CC=C1)C(=O)OCC[N+](C)(CC)CC.[Br-...,1
342514,CHEMBL487984,44589643,C23H27BrN4O3S,519.458,3-[1-[1-(benzenesulfonyl)piperidin-4-yl]piperi...,C1CN(CCC1N2C3=C(C=C(C=C3)Br)NC2=O)C4CCN(CC4)S(...,C1CN(CCC1N2C3=C(C=C(C=C3)Br)NC2=O)C4CCN(CC4)S(...,InChI=1S/C23H27BrN4O3S/c24-17-6-7-22-21(16-17)...,ZYKHMDFAAIQNQH-UHFFFAOYSA-N,3.4,1,5,"[15245973, 3827191, 2187645, 8872137, 2852937,...",C1CN(CCC1N2C3=C(C=C(C=C3)Br)NC2=O)C4CCN(CC4)S(...,1


In [16]:
MAP4 = MAP4Calculator(dimensions=1024)
def calc_map4(SMILES):
    mol = Chem.MolFromSmiles(SMILES)
    map4 = MAP4.calculate(mol)
    return np.array(map4)

In [19]:
glass["MAP4"] = glass["Canonical SMILES"].parallel_map(calc_map4)

RDKit WARNING: [18:21:27] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:28] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:28] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:37] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:49] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:49] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:50] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:21:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:22:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:22:13] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:22:18] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [18:22:19] WARNING: not r

In [95]:
fps = glass["MAP4"].tolist()
fps_ = []

for fp in fps:
    fp_ = tm.VectorUint(fp)
    fps_.append(fp_)

lf = tm.LSHForest(1024, 32)
lf.batch_add(fps_)

lf.index()

In [96]:
cfg = tm.LayoutConfiguration()

# config
cfg.node_size = 1 / 60
cfg.mmm_repeats = 2
cfg.sl_extra_scaling_steps = 50
cfg.k = 30
cfg.sl_scaling_type = tm.RelativeToAvgLength
# Compute the layout

x_, y_, s, t, gp = tm.layout_from_lsh_forest(lf, cfg)
tm_layout = {"x": list(x_), "y": list(y_), "s" : list(s), "t" : list(t)}

In [164]:
tmp = glass.copy()

idx = tmp[tmp["Molecular Weight"] >= 1000].index
tmp.loc[idx, "Molecular Weight"] = 1000

idx = tmp[tmp["Hydrogen Bond Acceptors"] >= 10].index
tmp.loc[idx, "Hydrogen Bond Acceptors"] = 10

idx = tmp[tmp["Hydrogen Bond Donors"] >= 5].index
tmp.loc[idx, "Hydrogen Bond Donors"] = 5

idx = tmp[tmp.XlogP >= 5].index
tmp.loc[idx, "XlogP"] = 5

idx = tmp[tmp.XlogP <= 0].index
tmp.loc[idx, "XlogP"] = 0

In [165]:
def create_label(row):
    smiles = row["Isomeric SMILES"]
    ID = row["InChI Key"].strip()
    link = f'https://zhanggroup.org/GLASS/ligand.cgi?id={ID}'
    label = f'{smiles}__<a target="_blank" href={link}>{ID}</a>'
    #label = smiles+"__"+NPAID+"__"+link
    return label

In [166]:
glass["label"] = glass.apply(create_label,1)

In [167]:
mw =  tmp["Molecular Weight"].to_list()
logp =  tmp["XlogP"].to_list()
hba =  tmp["Hydrogen Bond Donors"].to_list()
hbd =  tmp["Hydrogen Bond Acceptors"].to_list()
smiles =  glass["Isomeric SMILES"].to_list()
labels = glass["label"].to_list()

In [168]:
M4 = glass["M4_ligand"].to_list()

In [175]:
custom_cmap = ListedColormap([ 'darkgray', "magenta"], name="custom")
groups = ["0-No", "1-Yes"]
labels_groups, groups = Faerun.create_categories(groups)

In [177]:
pwd

'/home/alice/Code/GPCR_GLASS_DB'

In [176]:
faerun = Faerun(view="front", coords=False, title='GLASS')#, clear_color="#ffffff",)
faerun.add_scatter("GLASS",{"x": tm.VectorFloat(tm_layout["x"]), "y": tm.VectorFloat(tm_layout["y"]),\
                                "c": [mw, logp, hba, hba, M4],\
                          "labels": labels}, has_legend=True, \
    colormap=["rainbow", "rainbow", "rainbow", "rainbow", custom_cmap], \
        point_scale=3, categorical=[False, False, False, False, True],\
        series_title=["Molecular Weight", "XlogP", "Hydrogen Bond Donors", "Hydrogen Bond Donors", "targets Human M4 receptor"], \
            max_legend_label=[">=1000", ">=5", ">=10", ">=5", None],\
                   min_legend_label=[str(min(mw)),"<=0", str(min(hba)), str(min(hbd)), None],\
                    legend_labels=[None, None, None, None, labels_groups])


faerun.add_tree("GLASS_tree", {"from": tm.VectorUint(tm_layout["s"]), "to": tm.VectorUint(tm_layout["t"])}, point_helper="GLASS", color="aaaaaa")
faerun.plot('GLASS', template = 'smiles')

/home/alice/Code/GPCR_GLASS_DB/GLASS.html